<center>
    <h1>Introduction to Machine Learning/Deep Learning<BR>
        ICCAD 2020</h1>
</center>
<p>

<center>
Claudionor N. Coelho Jr - Palo Alto Networks, California, USA<br>
Sioni Summers - CERN, Geneva, Switzerland<BR>
Vladimir Loncar - CERN, Geneva, Switzerland<BR>
Jennifer Ngadiuba - CERN, Geneva, Switzerland<BR>
Thea Aarrestad - CERN, Geneva, Switzerland
</center>

<p>
This Jupyter Notebook was modified from Jupyter Notebook that was created for ICCAD 2019 by Claudionor N. Coelho Jr and Manish Pandey.
    
Participants familiar with Machine Learning and Deep Learning networks may skip this section of the Workshop.
    


## Introduction

In this section, we will present the basics of machine learning and deep learning so that you can follow up this material and learn even more in the future.

You probably have heard about Artificial Intelligence, Machine Learning and Deep Learning. How are they related? In Figure 1, we present a diagram showing their relation. Artificial Intelligence comprises of enhancing machines to exhibit some form of intelligence. Machine Learning is a branch of artificial intelligence with the property that machines can learn patterns from examples, and make decisions based on the patterns it has learned. In machine learning, the data is represented by features, which can be raw data or a function of the data (i.e. derivative of some variable). In several cases, features are computed from raw data. deep learning is a branch of machine learning, where we let the machine learn the features from raw data.

<center>
<figure>
  <img src="files/introduction.png" alt="Artificial Intelligence/Machine Learning/Deep Learning" style="width:50%">
  <figcaption>Figure 1 - Artificial Intelligence/Machine Learning/Deep Learning </figcaption>
</figure>
</center>

In very simple terms, we are trying to learn a function or process that takes $N$ variables represented by $x$ and convert it into $M$ outputs represented by variable $y$.

$$
y = f(x)
$$

Since we do not know a priori function $f$, we try to estimate it using some form of approximation by sampling $x$ from real world experiments.

If both $x$ and $y$ are available from our samples, we use supervised learning. In supervised learning, we supply to the learning algorithm both $x$ and corresponding $y$. If the result is a continuous function, we call the learning process a _regression_. If the output function we are trying to learn is discrete, we call the learning process a _classification_.

In several cases, we have a very large amount of data samples, but they have not been previously labeled (i.e. we have $x$ but not $y$), for example, when we have a large number of images, but we do not have a description of what is contained in the images. However, we still want to learn the structure of the data. In such cases, we can use unsupervised learning techniques. Some of the algorithms for unsupervised learning are k-means clustering, principal component analysis, and autoencoders.

This workshop is divided in the following sections. First, we will introduce you on how to read and prepare the data to use machine learning and/or deep learning. We will follow that with a brief introduction to linear regression techniques, logistic regression and the need to go deeper with deep convolutional networks. We will then present you with CNNs (Convolutional Neural Networks) and RNN (Recurrent Neural Networks).

## Data Preparation

Whenever you are going to use machine learning or deep learning, you will spend most of the time preparing and visualizing data.

This section is divided as follows. How to read datasets, most notably CSV (comma separated values), how to visualize data sets, and how to use principal component analysis to reduce the dimensionality of data.

We will first import the libraries we will use throughout this workshop.

In [ ]:
import os

import csv
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn import decomposition
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, MDS
import seaborn as sns

%matplotlib inline

np.random.seed(0)

### Reading MNIST Dataset

We will first read in MNIST dataset in this Workshop, and we will perform data preparation by visualizing this dataset, and by performing scaling of the input variables.

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

_, height, width = x_train.shape

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

It is always a good idea to visualize the data or a sample of the dataset in case there are too many samples, so that we can see some emerging patterns upfront.

In [ ]:
# let's visualize a random image and print its label
index = np.random.randint(len(x_train))
plt.title(str(y_train[index]))
plt.imshow(x_train[index], cmap='gray')
plt.show()

The first operation we perform on this dataset is to convert the output from a single output to a categorical output.

In this initial model, we will be using Dense networks, so the input shape we will be using has the shape $(B, H*W)$, where $B$ is the number of images on our training set, $H$ is the height of the image, $W$ is the width of the image, so we will collapse the height and width of the image. Later on, when we use convolutional networks, we will reshape the input image as $(B, H, W, C)$, where $B$, $H$ and $W$ are the same as before, and $C$ is the number of channels. So, for a black and white image, $C = 1$.

In [ ]:
# we have to convert the output to categorical data

y_train_c = to_categorical(y_train, num_classes=n_classes)
y_test_c = to_categorical(y_test, num_classes=n_classes)

# for this first experiment, we want to use a dense network

x_train = x_train.reshape((x_train.shape[0], height * width)).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], height * width)).astype(np.float32)

### Scaling Variables

When the dimension of $x$ is greater than 1, i.e. $N > 1$, it is often the case that variables have a very different ranges, as we have seen in the wine dataset, which may impact the analysis of the results, as we will see later. For example, when computing the correlation matrix, if variable ranges are too different, we may run into stability issues. 

Without going too much in the discussion at this time on why we need to scale the variables, we will give you a brief intuition on the subject.  In Figure 1, suppose we move from bottom to top on the y-axis of the ellipsis on the left. You can easily see that doing a full swing on the vertical axis corresponds only to going half way on the horizontal axis. On the other hand, on the circle, a full swing on circle on the horizontal axis of the circle corresponds to the same full swing on the vertical axis.

When we start running iteractive algorithms to train the network, if variables are unscaled, and as a result have different ranges between the minimum and maximum allowed values, we run the risk of doing a small swing on one of the axis, but performing a full swing on the other axis, which may take the second variable outside allowed ranges. As a result, we may reach either suboptimal results, or even run into numeric stability problems.

<center>
<figure>
  <img src="files/circles.png" alt="Unscaled and scaled 2d relations" style="width:50%">
  <figcaption>Figure 1 - Unscaled and scaled 2d relations </figcaption>
</figure>
</center>

There is no rule of thumb, and some people prefer to scale a variable using the mean and standard deviation, but a lot of times, it is good to keep variables scaled between $[-0.5,0.5]$, $[-1,+1]$ or $[0,1]$.  If you have any variable $x_i$, to scale this variable between 0 and +1, we can use the following formula.

$$
x_i' = \frac{x_i - \mathtt{min}(x_i)}{\mathtt{max}(x_i) - \mathtt{min}(x_i)}
$$

In case we want to scale variable $x_i$ between -0.5 and +0.5, we need to make the center point between $\mathtt{max}(x_i)$ and $\mathtt{min}(x_i)$ to be 0.

$$
x_i' = \frac{x_i - \mathtt{min}(x_i)}{\mathtt{max}(x_i) - \mathtt{min}(x_i)} - 0.5
$$

The input images can be scaled between 0 and 1 using sklearn preprocessing capability:

In [ ]:
x_scaler = MinMaxScaler(feature_range=(-0.5,0.5))
x_scaler.fit(x_train)

x_train = x_scaler.transform(x_train)
x_test = x_scaler.transform(x_test)
print(pd.DataFrame(x_train).min()[350], pd.DataFrame(x_train).max()[350])

Using sklearn preprocessing capability is very convenient because it enable us to quickly recover the original data.

In [ ]:
example = x_scaler.inverse_transform(x_train)
print(pd.DataFrame(example).min()[350], pd.DataFrame(example).max()[350])

Note that the original limits of any point in MNIST are 0 and 255, i.e. they should fit into an 8-bit number.

### Visualizing High Dimensional Datasets

There are several techniques to visualize high dimensional datasets. 

It is always good to visualize the histogram distribution of the data.  We can visualize a flattened view of all pixels of our training set, or visualizing the histogram of a given pixel.  In this case, let's visualize a pixel in the middle of the image, as most of the pixels in the corner are dark (have value 0).

In [ ]:
plt.hist(x_train[:, 350])
plt.show()

We have shown previously that we can plot the histogram of each of the variables to understand how well distributed the samples are.

We can plot each $x_i$ variable and output variable $y_k$ in a 2d plot.  We can also combine several of the input variables to a 2d space, for example, uing PCA (Principal Component Analysis) or SVD (Singular Value Decomposition), and plot them. This is specially interesting if $y_k$ is a discrete variable, as we can map each class to a different color.

There are other approaches beyond PCA and SVD, for example, we can use use a Linear Discriminant Analysis (LDA), which uses the labels to try to separate the classes instead of spreading the input dataset, thus providing better results in this case.

In [ ]:
def resample(xv, yv, sample_size=500):
    """
    Obtains a random sample (xv, yv) of sample_size values.
    """
    
    np.random.seed(0)
    xv = xv.copy()
    yv = yv.copy()
    
    sample = np.arange(xv.shape[0])

    np.random.shuffle(sample)

    sample = sample[0:sample_size]

    xv = xv[sample]
    yv = yv[sample]
    
    return xv, yv

def plot_visualize(xv, yv, n_classes=2, centroid=None):
    """
    Plots 2d values for each color given by the class of yv.
    """
    
    if xv.shape[1] == 1:
        return
    for _class in range(n_classes):
        mask = (yv == _class)
        
        # if we do not have any samples of mask, there is nothing to plot
        if np.sum(mask) == 0:
            continue

        plt.plot(xv[mask, 0], xv[mask, 1], 'o', label="class = {}".format(_class))
        
    if centroid is not None:
        for _class in range(n_classes):
            plt.plot(centroid[_class, 0], centroid[_class, 1], 'x', color="k")

    plt.legend(loc="best")
    plt.show()

In [ ]:
USE_LDA = 1

xv, yv = resample(x_train, y_train, sample_size=1000)
n_classes = 10

if USE_LDA:
    lda = LinearDiscriminantAnalysis(n_components=2)
    lda.fit(xv, yv)
    transform = lda.transform
else:
    pca = decomposition.PCA(n_components=2)
    pca.fit(xv)
    transform = pca.transform

plot_visualize(xv=transform(xv), yv=yv, n_classes=n_classes)

<a href="#maaten_hinton_tsne">Maaten et al</a> suggested using TSNE to visualize high-dimensional data, but if the dimensionality of the data is > 50, we should first use PCA or SVD to reduce it to 50. We are presenting the code here.  Even if the reader browse through the help on [manifold embedding](http://scikit-learn.org/stable/auto_examples/manifold/plot_compare_methods.html#sphx-glr-auto-examples-manifold-plot-compare-methods-py), the reader will see several different examples on how to fold multi-dimensional space into 2d space.

In [ ]:
xv, yv = resample(x_train, y_train, sample_size=1000)
plot_visualize(xv=TSNE(n_components=2).fit_transform(xv), yv=yv, n_classes=n_classes)

### Summary

In this section, we showed how to read MNIST dataset, and we then showed informally the importance of scaling variables using sklearn preprocessing pipeline, and showed how to reduce the dimensionality of the problem by using PCA, LDA and TNSE.

### Train-Validation-Test Set

For complex problems, the number of samples we have is far from complete, so we want to know how our model behaves when it estimates the value of the output function (predict) with new data.

Usually, we separate our dataset into three sets. 

The train set is the set we will use to train the model. The validation set is a set that helps us identify when we have trained the model enough (to avoid the model from learning too well how to beat the training set, but not some data it has not seen before - called overfitting), and to tune the training for hyperparameters (such as the model architecture, batch size, number of features  or variables, learning rate of the gradient descent algorithm, etc). The test set is the final test set where we will give a performance metric, and it should not be mixed with the training and validation sets.

Minimally, the validation and the test set should come from the same distribution.  It is not uncommon for people to use the same validation and test set, but they should not be the same in general, as we would be fine tuning the model for the validation set, and we cannot determine how the model will behave when we present new data it has not seen before.

We will use Keras to split the train and validation test in the following runs, by using the 90% of our dataset to belong to the training set, and 10% of our dataset to belong to the validation set.  Usually, using validation and test sets of 10,000 cases for very large training sets is enough.  For smaller training sets, we need a larger percentage of the training set allocated as validation and test sets.

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=42)
y_train_c = to_categorical(y_train, num_classes=n_classes)
y_valid_c = to_categorical(y_valid, num_classes=n_classes)
print(x_train.shape[0], "training images", x_valid.shape[0], "validation images")

Now, we will use Keras Dense model to try to classify the images. Do not worry if you do not understand every part of the code, as it will be explained later.

We are using Adam to train this network, and we will use categorical crossentropy as the loss function.

In [ ]:
model = Sequential(name="sequential")
model.add(Dense(n_classes, name="d2", activation="softmax", input_shape=x_train.shape[1:]))

model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

#### Dense Layer

A Dense layer performs the following operation $y = f(x) = W x + b$, where $W$ and $b$, so the function computing a cutting plane for each one of the output classes.

#### Multiclass Classification

When we are classifying a variable that may take several different values such as in the MNIST case (let's call each output variable $y_m$ to have $C_m$ classes), we have two options. We can create $C_m$ binary variables and label each output $y_m == m$ to be 1, and 0 otherwise; or we can use a $\mathtt{softmax}$ function. A softmax function performs the operation $\mathtt{softmax}(z) = \frac{e^{z_j}}{\sum_j e^{z_j}}$. This last option is what we will chose in the model defined above.

Let's see how this model train.

In [ ]:
history = model.fit(
    x_train, y_train_c, batch_size=64, epochs=30,
    validation_data=(x_valid, y_valid_c),
    shuffle=True, verbose=True)

In [ ]:
plt.plot(history.history['accuracy'], label="acc")
plt.plot(history.history['val_accuracy'], label="val_acc")
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="best")
plt.show()

plt.plot(history.history['loss'], label="loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="best")
plt.show()

From this graph, you can see that we should expect our model to respond to a new dataset with the same statistical distribution as our validation set with an accuracy close to 93%.

In addition, in the normal case, we should expect the validation loss to be higher than the training loss, which is not the case here. There are several explanations for that, and usually this can be explained by removing any randomization that occurs during the training process, such as it is the case of shuffling the input set that randomly samples the training set into chuncks of _batch\_size_. 

Finally, beyond looking for randomizations happening inside the training process, any oscillation on the training loss is also an indication we may have a learning rate that is too big.

Usually we use the validation set to tune hyperparameters such as learning rate, loss function and other hyperparameters such as network architecture.

## Going Deeper

Let's consider we are designing a ML model that classifies breeds of dogs, so that $x$ is an image of a dog, and $y$ is its breed. In Figure 2, we can see that our function $f$ maps images into breeds of dogs classes.

<center>
<figure>
  <img src="files/DogClassifier.png" alt="Dog Classifier" style="width:60%">
  <figcaption>Figure 2 - Dog Classifier </figcaption>
</figure>
</center>

The problem comes when our pictures are not as clean as the ones presented in Figure 2. In Figure 3, we can see that it is hard to define how our classifier should operate.

<center>
<figure>
  <img src="files/TwoDogs.png" alt="Two Dogs" style="width:60%">
  <figcaption>Figure 3 - Hard Classification Problem </figcaption>
</figure>
</center>

Because it is hard to define the boundaries of the dog we want to classify, we try to extract features from the image, like, a dog has two eyes, two ears, one mouth, one nose, the fur may have a color (black, brown, beige, etc), so that in Figure 4, if we feed a picture and we are able to extract the features of the dog for which we want to know the breed, the answer is simple, as given in Figure 4.

<center>
<figure>
  <img src="files/Terrier.png" alt="Feature based ML" style="width:70%">
  <figcaption>Figure 4 - Feature Based Machine Learning </figcaption>
</figure>
</center>

With enough testcases, we can improve our system to detect objects that it has not seen before, such as the dog in Figure 5.

<center>
<figure>
  <img src="files/Chiwawa.png" alt="Dog Detection" style="width:60%">
  <figcaption>Figure 5 - Chiwawa Detection </figcaption>
</figure>
</center>

Unfortunately, no matter how hard we engineer features for our model, it will never be able to  cover all possible variations, so that even the simple muffin of Figure 6 will be able to  fool the ML system, as it seems to have two eyes, two ears, a mouth, a nose, it is beige, and it will probably be mistakenly identified as a Chiwawa.

<center>
<figure>
  <img src="files/Muffin.png" alt="Dog Detection" style="width:20%">
  <figcaption>Figure 6 - Muffin </figcaption>
</figure>
</center>

What we need is a network that can learn not only how to classify features but to extract features. Because a human can think of a limited number of features, this network should be able compute several features and combine them layer after layer. The initial layers will detect simple features, perhaps slanted lines, with more complex features being detected later on. 

<center>
<figure>
  <img src="files/DeepNetwork.png" alt="DN" style="width:60%">
  <figcaption>Figure 7 - Deep Network trained on ImageNet from <a href="#zeiler_fergus">Zeiler and Fergus 2013</a></figcaption>
</figure>
</center>

So, in summary, we want to embed the function that generates the features $x$ into our ML model, for some raw data $r$ so that $x = g(r)$.

What Figure 7 represents is an embedding of the feature extraction $x = g(r)$ into our network model, so that our network becomes more robust and immune to changes. This new model now has two parts: a feature extraction part ($x = g(r)$) and a feature classifier $y = f(x)$.  We call this network a Deep Neural Network.

There are caveats to this new model.

1. End-to-end network may be harder to converge than breaking the network into smaller sub-problems;

2. We may know beforehand some relation between $r$ and $y$ (e.g. quadratic on some input raw variables), and as a result, we should feed this information to the network because the network model is able to abstract a non-linear behavior by approximating the function by parts. So, it will always be more precise to use non-linear relations instead of relying on the network to detect them (and it will be more expensive as well).

As a result, we should always carefully look at 1 and 2 when we decide to use a deep neural network.

## Deep Convolutional Neural Networks

In this section, we present main layers of deep convolution neural networks, and how they are connected.

### Convolution

If we consider a dense layer with $N_i$ inputs and $N_o$ outputs, we have $F_o (F_i + 1)$ parameters. This number may seem excessive if we have 100 inputs and 100 outputs, in which case we would have only for this layer 10100 parameters to be trained.  It is usually the case that an object of interest may be anywhere in the image, and as a result, most of these parameters will be attempting to detect the same feature (e.g. the face of a dog), which means it is desirable to have translation invariance of the feature being detected.

So, we can imagine that instead of performing operations on all of the inputs, it makes sense for images to perform local operations on the inputs, e.g. high pass, low pass, edge detection, corner detection, etc, and then compose the pieces together to obtain.  In Figure 9, we can see that we are detecting simple relations in the first layers, and  concatenating or combining the results for the next layers.

For example, if we are trying to detect wheels, we can first detect the edges of the wheels, and then combine the slopes to find a wheel, as presented in Figure 8.

<center>
<figure>
  <img src="files/Wheel-Detection.png" alt="Wheel-Detection" style="width:60%">
  <figcaption>Figure 8 - Composable Wheel Detection</a></figcaption>
</figure>
</center>

This filter has size $(K_h, K_w)$, and as a result, and for an input matrix of size $(I_H, I_W, I_C)$ where the triplet represents the height, width and the number of features (also called number of channels or filters, for example, for a RGB image, $I_C = 3$), and output matrix of size $(O_H, O_W, O_C)$, we will have $(K_h K_w I_C + 1) O_C$ parameters (where the 1 comes from the bias term), as we one filter ranging over all input channels for each output channel $O_C$.

Figure 9 represents the amount of computation that we need to perform to compute the convolution for each output pixel of $(O_H, O_W, O_C)$, taken from [machinethink.net/blog/googles-mobile-net-architecture-on-iphone](machinethink.net/blog/googles-mobile-net-architecture-on-iphone).  This figure assumes $I_C = 3$, $K_h = K_w = 3$ (we usually have square filters, normally $3 \times 3$) and $O_C = 1$.

<center>
<figure>
  <img src="files/Conv2D.png" alt="Conv2D" style="width:30%">
  <figcaption>Figure 9 - Convolution 2D</a></figcaption>
</figure>
</center>

When comparing the number of operations between a dense operation and a convolution operation, we have to assume $F_i = I_H I_W I_C$ and $F_o = O_H O_W O_C$, so ratio between the number of parameters in a dense operation and the number of parameters in a convolution operation is reduced by $\approx \frac{F_i F_o}{K_h K_w I_C O_C} = \frac{I_H I_W I_C O_H O_W O_C}{K_h K_w I_C O_C} = \frac{I_H I_W O_H O_W}{K_h K_w}$ if we ignore the bias term for the dense and Convolution operations.

The other question the reader may be asking is how many multiplication operations are we performing for both the dense and the convolution layer.  If you just assume a simple multiplication algorithm that does not use <a href="#lavin_gray">Winograd Convolution</a>, we have the following approximate number of multiplication operations.

- Multiplications for dense operations: $F_o F_i = I_H I_W I_C O_H O_W O_C$
- Multiplications for convolution operations: $K_h K_w I_C O_H O_W O_C$

The convolution operation has three other parameters we need to consider. First, the _stride_ , which is the offset for the next filter operation on the input matrix. Second, the _padding_ , which is usually set to _valid_ or _same_ , determining whether we should consider the input matrix to be filled with 0's to perform the filtering on the edges, or whether we should ignore the filters on the edges, and only perform the operations on filters that fully enclose the inputs.  Third, the _dilation_ . If greater than 1, the filter is not applied to consecutive pixels, but to pixels that are appart by this parameter.

Based on the stride $S$ and padding $P$, we can represent the relation between the input matrix and the output matrix as:

$$
O_H = \left\lfloor \frac{I_H - K_h + 2 * P}{S} \right\rfloor + 1,
O_W = \left\lfloor \frac{I_W - K_w + 2 * P}{S} \right\rfloor + 1,
$$

If we disregard the padding, the output can be computed as:

$$O[o_c][o_h][o_w] = \mathtt{bias}[o_c] + \sum_{k_h} \sum_{k_w} \sum_{i_c} I[i_c][S * o_h + k_h][S * o_w + k_w] * W[k_h][k_w][o_c][i_c]$$

The reader may be thinking that we only have benefits by using an convolution operation instead of a dense operation. In fact, the number of multiplications is indeed reduced, but at an expense to perform more memory operations to read an element of the input matrix, as an element $I[i_c][i_h][i_w]$ will be read $K_h K_w$ times from the memory, as it is involved in the computation of that many filter operations.

Another advantage of convolution operations is that it provides translation invariance. By applying the same filter everywhere on the input tensor, a convolution operation detects objects independent where the object is located in the image.

In Keras, the Convolution operation is invoked as the following function, whose may parameters are included here.

```python
keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid',
                    dilation_rate=(1, 1))
```

### Separable Convolutions

As one can see from the previous definition, a convolution operation is very good for images, but the number of multiplications may still be very large. Let's take for example the formula $K_h K_w I_C O_H O_W O_C$. This formula can be interpreted as for each output pixel of $(O_H, O_W, O_C$, we need to compute a filter of size $(K_h, K_w)$ for each input channel $I_C$.

The idea of separable convolutions is to replace this matrix computation by two matrix operations. In the first one, called Depthwise convolution, we perform the filtering operation of size $(K_h, K_w)$ from the input matrix $(I_H, I_W, I_C)$ into the intermediate matrix $(O_H, O_W, I_C)$, so that we only compute the output for the same channel at this stage (note that the intermediate matrix has the same number of channels as the input matrix. The resulting computation is presented as follows.

$$D[i_c][o_h][o_w] = \sum_{k_h} \sum_{k_w} I[i_c][S * o_h + k_h][S * o_w + k_w] * W_D[k_h][k_w][i_c]$$

In the second operation, called Pointwise convolution, we perform a weighted average on the input channels for each output channel.

$$O[o_c][o_h][o_w] = \mathtt{bias}[o_c] + \sum_{i_c} D[i_c][o_h][o_w] * W_P[o_c][i_c]$$

This Separable convolution is represented graphically in Figure 10, for $I_C = 3$, $K_h = K_w = 3$ and $O_C = 1$, also taken from  [machinethink.net/blog/googles-mobile-net-architecture-on-iphone](machinethink.net/blog/googles-mobile-net-architecture-on-iphone).

<center>
<figure>
  <img src="files/SeparableConv2D.png" alt="SeparableConv2D" style="width:50%">
  <figcaption>Figure 10 - Separable Convolution 2D</a></figcaption>
</figure>
</center>

What is the reduction on the number of multiplications from a normal convolution? Recall that the number of multiplications on a normal convolution is $K_h K_w I_C O_H O_W O_C$. The Depthwise convolution has $K_h K_w I_C O_H O_W$ operations, and the Pointwise convolution has $O_H O_W I_C O_C$ operations, so the reduction on the number of multiplications is $\frac{K_h K_w I_C O_H O_W + O_H O_W I_C O_C}{K_h K_w I_C O_H O_W O_C} = \frac{1}{O_C} + \frac{1}{K_h K_w}$.  So, for an operation with 200 output layers and a filter of size $3 \times 3$, we have  roughly only 11% of the number of multiplications when compared with a normal convolution. 

The reader should bear in mind that a separable convolution is an approximation for the full convolution operation, and as such, there are problems for which a separable convolution is not a good approximation.

In Keras, a separable convolution operation can be invoked by:

```python
keras.layers.SeparableConv2D(filters, kernel_size, strides=(1, 1), padding='valid',
                             dilation_rate=(1, 1))
```

### Activation

We have shown in the previous sections and also in Figures 9 and 10 that we would like to cascade a number of dense or convolution in order to detect higher level objects such as wheels, people, etc. The good news is that we can do that by cascading a number of such operations.  The bad news is that cascading convolutions or dense operations without a non-linearity in between can be collapsed by a single operation. 

Remember that we can represent a dense or convolution operation by a matrix multiplication $y = Ax$ without loss of generality. Now assume we are cascading two consecutive operations without a non-linearity between then. Then, we can represent the output as $y = A_2 (A_1 x) = (A_2 A_1) x = A x$, using the distributive property of matrix multiplication. This shows that without a non-linear activation layer, we can trivially collapse all of the dense or convolution operations into a single matrix operation.

There are several activation layers that can be used in deep learning. For intermediate layers, $\mathtt{relu}$, $\mathtt{leakyrelu}(x, \alpha) = (x \geq 0)?x:\alpha x$ and $\mathtt{relu6}(x) = \mathtt{clip}(x, 0, 6)$ are the most common for intermediate operations, although $\mathtt{relu}$ and $\mathtt{relu6}$ have the drawback of discarding all the computed information if $x < 0$. For that reason, <a href="#mobilenetv2">Sandler et al</a> proposed the use of bottlenecks to reduce the number of channels for the next operation by performing a linear combination of the channels (which can be implemented by a Conv2D operation where the filter size is (1,1)).

For the final classification operation, it is common to use $\mathtt{softmax}$ for multi-class classification and $\mathtt{sigmoid}$ for single or two-class classification problem. Of course if we are doing a regression or continuous function approximation, the final layer will not contain an activation operation.

In Keras, the activation operations can be invoked as presented below, where $\mathtt{activation}$ can be one of the strings: "relu", "softmax", "sigmoid", "tanh", among others. For $\mathtt{leakyrelu}$, we have to invoke it separately as it requires an additional parameter. Finally, $\mathtt{relu6}$ does not have a native representation in Keras, but it can be computed (as seen in the definition of MobileNetv2 in Keras by the function below.

```python
import tensorflow.keras
import tensorflow.keras.backend as K

def relu6(x):
  return K.relu(x, max_value=6)
  
keras.layers.Activation(activation)
keras.layers.Activation(relu6)
keras.layers.LeakyReLU(alpha=0.3)
```


### Batch Normalization

We spent a great deal of time in the beginning discussing why we should scale the inputs. Now, if we have two consecutive dense or convolution operations, shouldn't we worry about normalizing the intermediate values as well?

Batch normalization computes for each channel from the previous operation the running mean $\mu_\cal{B}$ and the standard deviation $\sigma^2_\cal{B}$ for a batch $\cal{B}$. Then, it computes two parameters $\gamma$ and $\beta$ to normalize the inputs as seen in Figure 11, taken from <a href="#batchnormalization">Ioffe and Szegedy</a>.

<center>
<figure>
  <img src="files/BatchNormalization.png" alt="BatchNormalization" style="width:40%">
  <figcaption>Figure 11 - BatchNormalization</a></figcaption>
</figure>
</center>

Batch normalization has an advantage as it can be merged in the weights and bias of the previous layer. In fact, if are using batch normalization we can turn off the bias in the convolution and dense layers by specifying the parameter $\mathtt{use\_bias=False}$ in these layers. If we do not set it, you will probably see that the bias is always 0 in these layers.  If you assumed the previous operation performed matrix multiplication $x_2 = W^{1,2} x_1$ and $x_3 = \mathtt{BatchNormalization}(x_2)$ where 1, 2 and 3 represent the operation levels (or the order in which the operations as cascaded) in the network model, the output of the batch normalization can be optimized by merging the two operations as:

$$
x_3 = \frac{\gamma (x_2 - \mu_{\cal{B}})}{\sqrt{\sigma^2_{\cal{B}} + \epsilon}} + \beta = 
      \frac{\gamma (W^{1,2} x_1 - \mu_{\cal{B}})}{\sqrt{\sigma^2_{\cal{B}} + \epsilon}} + \beta = \left( \frac{\gamma W^{1,2}}{\sqrt{\sigma^2_{\cal{B}}}} \right) x_1 +       
\left( \beta - \frac{\gamma \mu_{\cal{B}}}{\sqrt{\sigma^2_{\cal{B}} + \epsilon}} \right) = W^{1,2}_{new} x_1 + b_{new}
$$
      
The first term being the new weights of the matrix, and the second term being the new bias.

Whenever we want to embed a network with convolution and/or dense operations so that it uses lower precision (for example, using $\mathtt{int8}$ to represent the floating point numbers as fixed point numbers with 8 bits), using batch normalization may pose some challenges, as we usually need to retrain the network with the new arithmetic, but before the batch normalization is merged with the previous operations.


In Keras, the batch normalization can be invoked as using the following interface, where $\mathtt{axis}$ is 0 for dense operations and convolution operations when channels appear first in the matrices, or -1 for convolution operations when the channels appear last.

```python
keras.layers.BatchNormalization(axis=-1)
```

### Dropout

Dropout is a regularization layer that provides a way to prevent the neural network from overfitting (<a href="#dropout">Srivastava et al</a>).  A dropout randomly sets a fraction $\mathtt{rate}$ of the inputs to 0, during training time.  Because of that, the network learns how to optimize the loss function by using only a subset of the inputs, making the network more adaptable.

<center>
<figure>
  <img src="files/Dropout.png" alt="Dropout" style="width:50%">
  <figcaption>Figure 12 - Oringinal network (left) and with Dropout of 50% of the nodes (right)</a></figcaption>
</figure>
</center>

Figure 12, shows two networks with cascaded dense layers. In the network on the right, we are only using 50% of the inputs in the intermediate layer to compute the output (dropout of 50%).

Because of dropout, during training, we are not training the inputs $x$ with expected value $\mathtt{E}[x]$ but instead $\mathtt{rate} * \mathtt{E}[x]$, as we will be dropping $\mathtt{rate}$ of the inputs. As a result, during inference, we have to adjust the statistics of the inputs to match that during training. In order to overcome this difference, deep learning frameworks scale up the values during training by $1/\mathtt{rate}$ so that you do not have to worry on what to do during inference.

When using dropout and batch normalization, we may have some problems though. Remember that in batch normalization, the inputs are adjusted according to the statistic properties of the batch, namely, the mean and the standard deviation. Since dropout randomly drops inputs from the operation, we are basically changing the statistic properties of the values for the following operations. As such, the user should avoid using batch normalization after we use dropout operations, according to <a href="#batchnormalization_and_dropout">Li et al</a>, or we need to adjust the inputs to the incoming batch normalization layers after dropout.

In Keras, dropout can be invoked as follows.

```python
keras.layers.Dropout(rate)
```

### Pooling

We have already seen that convolution operations reduce the dimensionality of the output tensor, especially if we use a stride greater than 1. Another way to provide dimensionality reduction is through pooling. Usually, pooling is interleaved with convolution layer, and we can perform max, average or random pooling on a window (typically 2x2) with a stride of 2, to guarantee a dimensionality reduction by 2.

Pooling provides immunity against noise, and depending on the filter, it can also provide rotation invariance.

In Figure 13,  taken from [http://cs231n.github.io/convolutional-networks/#pool](http://cs231n.github.io/convolutional-networks/#pool), we can see the effect of a max pooling operation on the inputs.   

<center>
<figure>
  <img src="files/MaxPooling2D.png" alt="MaxPooling2D" style="width:50%">
  <figcaption>Figure 13 - MaxPooling2D</a></figcaption>
</figure>
</center>

How can pooling provide rotation immunity? Suppose we are computing four different types of filters to detect if a line is in horizontal, veritical, inclined to the right or left.  If we feed the image that maximizes that filter, and perform a max pooling operation, the result will be exactly the filtered result that best matches the rotation of the input.

<center>
<figure>
  <img src="files/HowMaxPoolingWorks.png" alt="HowMaxPoolingWorks" style="width:50%">
  <figcaption>Figure 14 - Rotation invariance in max pooling</a></figcaption>
</figure>
</center>

Finally, global average pooling is an operation that performs the average over the width and height of the input matrix, returning a vector with the size of the input features, i.e. $O[i_c] = \frac{1}{I_H I_W} \sum_{i_h} \sum_{i_w} I[i_c][i_h][i_w]$.  Global average pooling is used in all convolutional networks (<a href="#all_conv">Springenberg et al</a>) to reduce the classification layer to the only the channels.

In Keras, pooling operations can be invoked as using the following interface.

```python
keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')
keras.layers.GlobalAveragePooling2D()
```

### Additional Convenient Operations

We are finally ready to create a model that uses deep learning. Before that, we need to talk about a few of the layers that were left but will be necessary in order to create a model.

```python
keras.layers.Reshape(target_shape)
keras.layers.Flatten()
keras.layers.Concatenation(axis=-1)
keras.layers.Add()
```

Reshape converts network from one shape into another shape, preserving the total number of elements of the input.  Reshape is usually used to convert from a flattened matrix to an image with height, width and channels.

Flatten converts an image with three dimensions (height, width and channels) to a single dimension. It is usually used when you want to extract the features using a convolutional network, and perform feature classification using dense network.


Concatenation and Add are used as merging operations.

Concatenation is usually used when you have two different parallel networks and you want to combine them in the dimension $\mathtt{axis}$.

Finally, Add is used when you want to merge the two different parallel networks by adding them together instead of concatenating outputs.

### Putting All Together - Building Deeper Layered Networks

We will show now how to put everything we have seen before to compose bigger deeper neural networks. We will start by the concept of a layer. Although keras call layers all the operations for historical reasons, and in fact we may have used the term layer interchangeably with operation in the previous sections, a layer is a "composable" unit with operations, and in most of the cases, it may contain a single dense or convolution operation, although in MobileNetV2, the authors have used a second convolution operation to provide a bottleneck operation.

So, in Figure 15, we can see what a normal layer looks like.  The current consensus is that a non linear activation should come after batch normalization, although some authors have suggested that batch normalization should come after the non linear activation.

<center>
<figure>
  <img src="files/Layer.png" alt="Layer" style="width:40%">
  <figcaption>Figure 15 - Composable operations forming a layer</a></figcaption>
</figure>
</center>

the layers can be composed in parallel or sequential to for a feature extraction network, as seen in Figure 16.

<center>
<figure>
  <img src="files/Network.png" alt="Network" style="width:60%">
  <figcaption>Figure 16 - Deep Learning Network</a></figcaption>
</figure>
</center>

There are some tendencies we have seen in small or new networks.

- Use separable convolution whenever possible for kernels bigger than 1x1,
- Use batch normalization in feature extraction layers to improve convergence,
- After dropout do not use batch normalization, as it shifts mean and variance between training and inference,
- We can use 1x1 convolution operations in the feature classification layers instead using dense operations, as presented by <a href="#all_conv">Springenberg et al</a>,
- Always start with a network without batch normalization and without dropout and add them later to see if results improve, especially because batch normalization will make quantization more difficult,
- If we use relu or relu6, we can use a convolution with filter size 1x1 without batch normalization and activation to reduce the number of features as part of the features will be 0's whenever they are negative,
- It is worth trying to use a stride=2 in the convolution operation instead of using pooling.

In this example, we will use the MNIST dataset to execute this exercise, which are samples of hand written digits stored in 28x28 gray scale matrix.

In [ ]:
model = Sequential()
model.add(Dense(50, input_shape=(height * width,)))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("softmax"))

model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(x_train, y_train_c, batch_size=64, epochs=20, shuffle=True,
                    validation_data=(x_valid, y_valid_c), verbose=True)

plt.plot(history.history['accuracy'], label="acc")
plt.plot(history.history['val_accuracy'], label="val_acc")
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc="best")
plt.show()

plt.plot(history.history['loss'], label="loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc="best")
plt.show()

metrics = model.evaluate(x_test, y_test_c, batch_size=64, verbose=1)
print(metrics[0], metrics[1])


You can see that the MNIST dataset is very easy to train and with a small network, we can already have a very good performance, which is already over 95% on both the validation and test sets.  However, we are using 40k parameters 
in our network. We will try now with a small convolutional network.

In [ ]:
x_train = x_train.reshape((x_train.shape[0], height, width, 1))
x_valid = x_valid.reshape((x_valid.shape[0], height, width, 1))
x_test = x_test.reshape((x_test.shape[0], height, width, 1))

kernel = (3, 3)
strides = (2, 2)

model = Sequential()
model.add(Conv2D(8, kernel, strides=strides, padding='same', input_shape=(height, width, 1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))
model.add(Conv2D(16, kernel, strides=strides, padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))
model.add(Conv2D(24, kernel, strides=strides, padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(n_classes))
model.add(Activation("softmax"))

model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(x_train, y_train_c, batch_size=64, epochs=10, shuffle=True,
                    validation_data=(x_valid, y_valid_c), verbose=True)

metrics = model.evaluate(x_test, y_test_c, batch_size=64, verbose=1)
print(metrics[0], metrics[1])


With 5x less parameters, we obtained the same performance, or 1% better. The reader may be asking why the first one seemed to train faster than the second one? Well, we only had a big matrix multiplication in the first network, and on the second one, we have smaller matrix multiplications. How many multiplications do we have in both cases? 

Finally, let us inspect some of the results to see how we are doing.

In [ ]:
p = model.predict(x_test)
labels = np.argmax(p, axis=-1)
for i in range(3):
    idx = np.random.randint(y_test.shape[0])
    plt.imshow(x_test[idx].reshape(28, 28), cmap="gray")
    plt.title("Predicted label is " + str(labels[idx]))
    plt.show()

### Summary

We have shown in this section the building blocks used to build deep neural networks with application to the classification of MNIST dataset. Of course, MNIST dataset is a very easy dataset to get high accuracy, and you should classify images from cifar10 in the rest of the time you have in this workshop.

Right now, for more difficult problems, networks such as <a href="#googlenet">GoogleNet</a> or <a href="#resnet">Resnet</a> are used, which contains many more layers, as seen in Figure 19. 

<center>
<figure>
  <img src="files/GoogleNet.png" alt="GoogleNet" style="width:80%">
  <figcaption>Figure 19 - GoogleNet</a></figcaption>
</figure>
</center>

In [ ]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

cifar10_labels = [
    "airplane", "automobile", "bird", "cat", "deer", "dog", "frog", 
    "horse", "ship", "truck"
]

print(x_train.shape)

for i in range(3):
    plt.imshow(x_train[i])
    plt.title(cifar10_labels[y_train[i][0]])
    plt.show()

# now it is your turn to create a network to classify cifar10 images
# remember cifar10 has input size (32, 32, 3)

# your code begins here

## Conclusions

In this section, we introduced the reader to the layers deep neural netowrks, and with this information, the reader should be able to create small deep neural networks.

We tried to steer away from statistical details, trying to focus on ML/DL as an optimization problem, showing the readers an intuition on why things are done in a certain way.

This should give you enough information to start using ML/DL, getting gratification from and and start looking for additional resources. 

## Bibliography


<a id="maaten_hinton_tsne"></a>[1] van der Maaten, L.J.P.; Hinton, G.E. Visualizing High-Dimensional Data
Using t-SNE. Journal of Machine Learning Research 9:2579-2605, 2008.

<a id="zeiler_fergus"></a>[2] Zeiler, M.D.; Fergus, R. Visualizing and Understanding Convolutional Networks. [https://arxiv.org/abs/1311.2901](https://arxiv.org/abs/1311.2901)

<a id="lavin_gray"></a>[3] Lavin, A.; Gray, S. Fast Algorithms for Convolutional Neural Networks. [https://arxiv.org/abs/1509.09308](https://arxiv.org/abs/1509.09308)

<a id="mobilenetv2"></a>[4] Sandler, M.; Howard, A.; Zhu, M.; Zhmoginov, A.; Chen, L.-C. MobileNetV2: Inverted Residuals and Linear Bottlenecks. [https://arxiv.org/abs/1801.04381](https://arxiv.org/abs/1801.04381)

<a id="batchnormalization"></a>[5] Ioffe, S.; Szegedy, C. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift [https://arxiv.org/abs/1502.03167](https://arxiv.org/abs/1502.03167)

<a id="dropout"></a>[6] Srivastava, N.; Hinton, G.; Krizhevsky, A.; Sutskever, I.; Salakhutdinov, R. Dropout: A Simple Way to Prevent Neural Networks from
Overfitting. Journal of Machine Learning Research, 15, 1929-1958, 2014 [http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf](http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)

<a id="batchnormalization_and_dropout"></a>[7] Li, X.; Chen, S.; Hu, X.; Yang, J. Understanding the Disharmony between Dropout and Batch Normalization by Variance Shift [https://arxiv.org/abs/1801.05134](https://arxiv.org/abs/1801.05134)

<a id="all_conv"></a>[8] Springenberg, J. T.; Dosovitskiy, A.; Brox, T.; Riedmiller, M. Striving for Simplicity: The All Convolutional Net
[https://arxiv.org/abs/1412.6806](https://arxiv.org/abs/1412.6806)

<a id="googlenet"></a>[9] Szegedy, C.; Liu, W.; Jia, Y.; Sermanet, P.; Reed, S.; Anguelov, D.; Erhan, D.; Vanhoucke, V.; Rabinovich, A. Going Deeper with Convolutions
[https://arxiv.org/pdf/1409.4842.pdf](https://arxiv.org/pdf/1409.4842.pdf)

<a id="resnet"></a>[10] He, K.; Zhang, X.; Ren, S.; Sun, J.;
Deep Residual Learning for Image Recognition
[https://arxiv.org/abs/1512.03385](https://arxiv.org/abs/1512.03385)

<a id="smallminibatches"></a>[11] Masters, D.; Luschi, C.; Revisiting Small Batch Training for Deep Neural Networks
[https://arxiv.org/abs/1804.07612](https://arxiv.org/abs/1804.07612)

<a id="radam"></a>[14] Liu, L., Jinag; H., Pengcheng He, W.; Xiaodong Liu, J.; Han, J.; On the Variance of the Adaptive Learning Rate and Beyond [https://arxiv.org/pdf/1908.03265.pdf](https://arxiv.org/pdf/1908.03265.pdf)